In [0]:
import numpy as np
import pandas as pd
import urllib.request as request
from pathlib import Path
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

page = request.urlopen('https://raw.githubusercontent.com/ShreyaGupta08/Lyrics-Generator/master/data/data.txt?token=AVvj6Rofb-Azo9OmR2gHhIu2y0yNH3Qkks5cpKPkwA%3D%3D')
doc = bytes(page.read())

In [0]:
words = doc.split()
words = words[ : int(0.1*len(words))]
vocab_size = len(set(words))
dic = {}
rev = {}
cnt = 0
for ix in set(words):
    dic[ix] = cnt
    rev[cnt] = ix
    cnt += 1
sequence = []
for ix in words:
    sequence.append(dic[ix])

In [0]:
import numpy as np
seq_len = 5
X = []
y = []
for ix in range(0, len(sequence)-seq_len, seq_len):
    X.append(sequence[ix:ix+seq_len])
    y.append(sequence[ix+seq_len])
    
X = np.array(X)
y = np.array(y)

In [0]:
seq_length = X.shape[1]
y = to_categorical(y, num_classes=vocab_size)

In [0]:
# define model
vec_size = 10
model = Sequential()
model.add(Embedding(vocab_size, vec_size, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 5, 10)             101600    
_________________________________________________________________
lstm_3 (LSTM)                (None, 5, 100)            44400     
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 10160)             1026160   
Total params: 1,262,660
Trainable params: 1,262,660
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
# fit model
model.fit(X, y, batch_size=128, epochs=200)
model.save('ep300.h5')

Epoch 1/200
108769/108769 [==============================] - 36s 331us/step - loss: 6.6338 - acc: 0.0396
Epoch 2/200
108769/108769 [==============================] - 34s 317us/step - loss: 6.3011 - acc: 0.0402
Epoch 3/200
108769/108769 [==============================] - 34s 310us/step - loss: 6.0937 - acc: 0.0489
Epoch 4/200
108769/108769 [==============================] - 33s 307us/step - loss: 5.8955 - acc: 0.0641
Epoch 5/200
108769/108769 [==============================] - 34s 309us/step - loss: 5.7411 - acc: 0.0737
Epoch 6/200
108769/108769 [==============================] - 33s 306us/step - loss: 5.6052 - acc: 0.0795
Epoch 7/200
108769/108769 [==============================] - 33s 307us/step - loss: 5.4825 - acc: 0.0842
Epoch 8/200
108769/108769 [==============================] - 33s 307us/step - loss: 5.3650 - acc: 0.0904
Epoch 9/200
108769/108769 [==============================] - 33s 307us/step - loss: 5.2426 - acc: 0.0969
Epoch 10/200
108769/108769 [===========================

In [0]:
inp = words[2000:2000+seq_len]
X_test = []
for ix in inp: 
    X_test.append(dic[ix])
X_test = np.array(X_test)
X_test.shape
X_test = X_test.reshape(1, -1)

In [0]:
seq = []
for ix in range(X_test.shape[1]):
    seq.append(X_test[0, ix])
len(seq)

5

In [0]:
for ix in range(0, 10):
    pred = model.predict(X_test)
    seq.append(int(np.argmax(pred)))
    X_test = np.array(seq[ix+1:]).reshape(1, -1)

In [0]:
song = []
for ix in range(len(seq)):
    song.append(rev[seq[ix]])

In [0]:
model.save('model.h5')

In [0]:
for ix in song:
  print(ix)

b'for'
b'you'
b'the'
b'one'
b'that'
b'i'
b'adore'
b'hold'
b'by'
b'a'
b'business'
b'man'
b'youve'
b'known'
b'me'


In [0]:
final_lyrics=''
for x in song:
  final_lyrics = final_lyrics + ' ' + str(x,'utf-8')
  
final_lyrics

' for you the one that i adore hold by a business man youve known me'